#### Autoencoader:
Autoencoders are a Feature learning technique. Autoencoders are a specific type of feedforward neural networks where the input is the same as the output. They compress the input into a lower-dimensional code and then reconstruct the output from this representation. The code is a compact **`summary`** or **`compression`** of the input, also called the ***`latent-space representation`***.

An autoencoder model contains two components:
- Encoader: An encoder that takes an image as input, and outputs a low-dimensional embedding (representation) of the image called **latent-space representation**. We can call this dimensionality reduction. The encoder can be made with-
    - Linear or non linear layers (earlier days)
    - Deep fully connected NN (Then)
    - RELU CNN (Currently we use this on images)

- Decoader: A decoder that takes the low-dimensional embedding(is also knoown as bottleneck) or feature vector, and reconstructs the image. The decoder can be made with same techniques we made the encoade.


<p align="center"><img width="50%" src="../images/auto-encoder.png" /></p>

***Cost Function:*** $L(\theta, \phi) =  \sum_{i=1}^{n}[x^{(i)}-f_{\theta}(g_{\phi}(x^{(i)}))]^2$
- $x^{i}=x$ 
- $g_{\phi}(x^{(i)}) = Z$ is the feature vector
- $f_{\theta}(g_{\phi}(x^{(i)})) = \bar{x}$ is the output

***The encoder is a conv layer while the decoder is deconv layer! Means Decreasing and then increasing.***

##### Usage:
* We can use this encoder we have to make a supervised model.
* The value of this it can learn a good feature representation to the input you have.
* A lot of times we will have a small amount of data to solve problem. One way to tackle this is to use an Autoencoder that learns how to get features from images and train your small dataset on top of that model.

##### Architecture Hyperparameter:
* Number of Layers: Determine the depth of the encoder and decoder networks.
* Number of Neurons per Layer: Decide the size of each layer in the encoder and decoder.
* Activation Functions: Choose activation functions (e.g., ReLU, sigmoid, tanh) for each layer.
* Loss Function: If the input values are in the range [0, 1] then we typically use crossentropy, otherwise we use the mean squared error.

##### ***Denoising Autoencoders::***
When we add noise(**Gaussian noise**) to the input image before passing it to encoader and encoder then generate a noise free orginal image. Then decoder reconstracts the image.

##### ***Sparse Autoencoders::***
When we regularize the autoencoder by using a **sparsity constraint** such that only a fraction of the nodes would have nonzero values, called active nodes. In particular, we add a penalty term to the loss function such that only a fraction of the nodes become active. This forces the autoencoder to represent each input as a combination of small number of nodes, and demands it to discover interesting structure in the data. Sparse autoencoders are often used for feature learning and dimensionality reduction.

#### Can we generate data from this encoder?
#### Variational Autoencoders (VAE):
VAEs introduce probabilistic modeling to autoencoders, treating the encoder's output as a probability distribution in the latent space.The encoder produces not only a mean and variance (or other parameterization) of the latent space but also samples from that distribution. The decoder then takes these samples and generates reconstructed data from them. VAEs are useful for applications like data generation, image synthesis, and learning disentangled representations. VAEs enable generative modeling, meaning they can generate new data samples by sampling from the latent space distribution.

***Encoder also called recognition/inference networks and decoder networks  also called generation networks.***

<p align="center"><img width="70%" src="../images/vae.png" /></p>

##### How to train the model VAE?
We get a smaple $Z$ from the encoder, $Z$ is the latent factors used to generate $X$ prior over $Z$, each of these latent factor attributes responsible for example .9 pose, how much smile. We want to estimate of the true parameters $\theta^*$ of this generative model. Simply, if we pass an input image x and encoder ecode this with multiple laten attributes based on, {smile:.9, gender: .9}

##### What is the problem of $P_{\theta}(x) = \int P_{\theta}(z)P_{\theta}(x |z)dz$?

Data likehood: $P_{\theta}(x) = \int P_{\theta}(z)P_{\theta}(x |z)dz$<br>
 -  $P_{\theta}(z)$: Simple Gaussian prior.
 - $P_{\theta}(x |z)$: Decoder neural network.
 - $\theta$: is the bias parameter

 This is `intarctable` because of computing $P(x\mid z)$ for every z.
 
Posterior density also [intractable](../math/probability.ipynb): $P_{\theta}(z\mid x) = \frac {P_{\theta}(z)P_{\theta}(x \mid z)}{P_{\theta}(x)}$  because of $P_{\theta}(x)$

Solution: In addition to decoder network modeling $p_{\theta}(x\mid z)$, define additional encoder network $q_{\phi}(z\mid x)$ that approximates $p_{\theta}(z\mid x)$.<br>
Will see that this allows us to derive a lower bound on the data likelihood that is tractable, which we can optimize.

![vae-2](../images/vae2.png)

(log) data likelihood-

$\log p_{\theta}(x^{(i)}) = E_{z\sim q_{\phi}(z\mid x^{(i)}) [\log p_{\theta}(x^{(i)})]}$   ($p_{\theta}(x^{(i)})$ Does not depend on z)<br>

$= E_{z} \;[\log {\frac { p_{\theta}(x^{(i)} \mid z)p_{\theta}(z)} {p_{\theta} (z\mid x^{(i)})} }]$    (Baye's theorem)<br>

$= E_{z} \;[\log {\frac { p_{\theta}(x^{(i)} \mid z)p_{\theta}(z) \times q_{\phi}(z\mid x^{(i)}) } {p_{\theta} (z\mid x^{(i)}) \times q_{\phi}(z\mid x^{(i)})} }]$    (Multiply by constant)<br>

$= E_{z}[\log p_{\theta}(x^{(i)}\mid z)]- E_{z}[\log{\frac {q_{\phi}(z\mid x^{(i)})} {p_{\theta}(z)}}]+ E_{z}[\log{\frac {q_{\phi}(z\mid x^{(i)})} {p_{\theta}(z\mid x^{(i)})}}]$    (Logarithms)<br>

$= E_{z}[\log p_{\theta}(x^{(i)}\mid z)]- D_{KL}(q_{\phi}(z\mid x^{(i)}) \parallel p_{\theta}(z))+ D_{KL}(q_{\phi}(z\mid x^{(i)}) \parallel p_{\theta}(z\mid x^{(i)}))$    (Logarithms) ....(1)<br>

here,
- ***$E_{z}[\log p_{\theta}(x^{(i)}\mid z)]$:*** 
    - **Decoder network** gives $ p_{\theta}(x\mid z)$, can compute estimate of this term given sampling
    - will generate new data.


- ***$D_{KL}(q_{\phi}(z\mid x^{(i)}) \parallel p_{\theta}(z))$:*** 
    - This KL term (between Gaussians for encoder and z prior).
    - Make approximate posterior distribution ($q_{\phi}(z\mid x^{(i)})$) close to prior ($p_{\theta}(z)$).


- ***$D_{KL}(q_{\phi}(z\mid x^{(i)}) \parallel p_{\theta}(z\mid x^{(i)}))$:*** 
    - $p_{\theta}(z\mid x)$ is intractable, however, we know KL-divergence always $\geq 0$

- ***$E_{z}[\log p_{\theta}(x^{(i)}\mid z)]- D_{KL}(q_{\phi}(z\mid x^{(i)}) \parallel p_{\theta}(z))$***: this is also called Lower Bound ($L(x_i, \theta, \phi)$).
    - Tractable lower bound which we can take gradient of and optimize! $(p_{\theta} (x\mid z))$ differentiable, KL term differentiable)
    - In training Maximize the lower bound. So , maximize $\theta , \phi = {arg \; max}_{\theta, \phi}\sum_{i=1}^{N}L(x^{(i)}, \theta , \phi)$

![vaes](../images/vae_3.png)

##### Pros and Cons:
Pros:
- Principled approach to generative models
- Allows inference of q(z|x), can be useful feature representation for other tasks

Cons:
- Maximizes lower bound of likelihood: okay, but not as good evaluation as
PixelRNN/PixelCNN
- Samples blurrier and lower quality compared to state-of-the-art (GANs)
